### Module and Function Instance

In [18]:
import requests
import pypyodbc
import time
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import urllib
import re
import pyodbc
import html5lib
import csv
from colorama import Fore, Style #print statement font

# # Parameter

Last_Round = 'MW_5'
Next_Week = 6
Upper_Week = 7

# 'replace' or 'append'
Data_Base_Push_Method = 'append'

# select sink database option ["Az SQL", "Bif_Database"]
sink = "Az SQL"


# #### Connect to database

# ### Az Sql
if sink == "Az SQL":
    SERVER_NAME = 'socceranalyticsgroup.database.windows.net'
    DATABASE_NAME = 'eplanalytics'
    USERNAME = 'socceranalyticsgroup'
    PASSWORD = 'Zador@63'

    driver= '{ODBC Driver 17 for SQL Server}'
    conn_str = (
        f"Driver={driver};"
        f"Server=tcp:{SERVER_NAME};"
        f"Database={DATABASE_NAME};"
        f"Uid={USERNAME};"
        f"Pwd={PASSWORD};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=no;"
        f"Connection Timeout=30;"
    )

    # Create a pyodbc connection object
    conn = pyodbc.connect(conn_str)

    # Create a SQLAlchemy engine object
    # engine = create_engine("mssql+pyodbc://", creator=lambda: conn)
    connection_url = f"mssql+pyodbc:///?odbc_connect={conn_str}"
    engine = create_engine(connection_url)

# ### Bif On Premise
if sink == "Bif_Database":
    SERVER_NAME = 'WS1\\WSBIF'
    DATABASE_NAME = 'SoccerAnalysis'
    USERNAME = 'WS1\\User'
    conn= pypyodbc.connect("""
        Driver={{SQL Server}};
        Server={0};
        Database={1};
        Trusted_Connection=yes;""".format(SERVER_NAME, DATABASE_NAME))

    engine = create_engine("mssql+pyodbc://@"+SERVER_NAME+"/"+DATABASE_NAME+"?trusted_connection=yes&driver=SQL+Server")



# # Renaming some club
#creating a class that inherit from the dictionary class for mapping of the name variation
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_values = {
    "Wolverhampton Wanderers": "Wolves",
    "West Ham United": "West Ham",
    "Tottenham Hotspur": "Tottenham",
    "Nottingham Forest": "Nott'ham Forest",
    "Newcastle United": "Newcastle Utd",
    "Manchester United": "Manchester Utd",
    "Brighton and Hove Albion": "Brighton",
    "Sheffield United": "Sheffield Utd"
}
mapping = MissingDict(**map_values)


# # Functions
# return list of all column in sqltable
def sourcedestcompare(sqltablename:str, pythondf):
    sql_columnlist= """  
    Select COLUMN_NAME from INFORMATION_SCHEMA.COLUMNS
    where TABLE_NAME = '{0}'

    """.format(sqltablename)
    
    df_sqlcolumnlist = pd.read_sql(sql_columnlist, conn)

    # remove the index column coming from database
    sqllist = df_sqlcolumnlist[1:].values.tolist()
    # convert list of list into list of strings
    sqllist = [''.join(ele) for ele in sqllist]
    
    dflist=pythondf.columns.values.tolist()
    
    # identify difference between sql table and pydataframe
    # what sql has but df doesnt contain
    temp = []
    for x in sqllist:
        if x not in dflist:
            temp.append(x)        
    print(temp)
    # what df contains but sql doesnt have
    temp1 = []
    for x in dflist:
        if x not in sqllist:
            temp1.append(x)        
    print(temp1)
#     return sqllist

# rename duplicate field from fbref.com
def renameduplicatecolumns(dlist, df: pd.DataFrame):
    for ele in dlist:
        cols = []
        count = 1
        for column in df.columns:
            if column == ele:
                cols.append(f'{ele}_{count}')
                count+=1
                continue
            cols.append(column)
        df.columns = cols

def add(x,y):
    result = x+y
    return(result)

In [4]:
def read_csv_to_array_of_tuples(csv_file):
    result = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header if exists
        for row in reader:
            # Convert the row into a tuple
            result.append(tuple(row))
    return result


## Function: player_record_to_database

In [5]:
def player_record_to_database(html_identifier, database_table):
    standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text, 'lxml')
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if 'squads' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    # url_to_remove = 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats'

    
    team_urls = [url for url in team_urls 
                 # if url != url_to_remove
                 # if url == "https://fbref.com/en/squads/19538871/Manchester-United-Stats"
                ]
    all_squad_record = []

    # List to store affected iterations
    error_iterations = []


    # replace tem_urls with below to limit the iteration for testing
    # (team_url_filter for team_url_filter in team_urls if team_url_filter == "https://fbref.com/en/squads/fd962109/Fulham-Stats")
    for team_url in team_urls:
        try:
            print(f"{team_url} processing")
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    
            
            data = requests.get(team_url)
            squad_record = pd.read_html(data.text, match = html_identifier)[0]
            squad_record.columns = squad_record.columns.droplevel()
            squad_record["Club"] = mapping[team_name]
            squad_record.drop(squad_record.tail(2).index, inplace=True)
            all_squad_record.append(squad_record)
            time.sleep(15)
            print(f"{Fore.GREEN}Table completed: {team_url}{Style.RESET_ALL}")
        except ValueError as e:
            print(f"{Fore.RED}Error processing {team_url}: {e}{Style.RESET_ALL}")
            error_iterations.append((html_identifier, team_url))  # Store the tuple
            continue  # Skip to the next iteration

    if error_iterations:
        print("\nThe following iterations encountered errors:")
        for html_id, url in error_iterations:
            print(f"{Fore.RED}  - html_identifier: {html_id}, URL: {url}{Style.RESET_ALL}")

    all_squad_record_df = pd.concat(all_squad_record)
    all_squad_record_df["Season"] = "2024/2025"

    if html_identifier == "Goalkeeping ":
        RepeatedColumn = ['Save%']
        renameduplicatecolumns(RepeatedColumn, all_squad_record_df)
    if html_identifier == "Advanced Goalkeeping ":
        RepeatedColumn = ['Att', 'Launch%', 'AvgLen']
        renameduplicatecolumns(RepeatedColumn, all_squad_record_df)
    if html_identifier == "Passing ":
        RepeatedColumn = ['Cmp','Att','Cmp%']
        renameduplicatecolumns(RepeatedColumn, all_squad_record_df)
        sourcedestcompare(database_table, all_squad_record_df)
    if html_identifier == "Standard Stats ":
        RepeatedColumn = ['Gls','Ast','xG','xAG']
        renameduplicatecolumns(RepeatedColumn, all_squad_record_df)
        sourcedestcompare(database_table, all_squad_record_df)
        all_squad_record_df = all_squad_record_df.drop(['PrgC','PrgP','PrgR'],axis=1)
    if html_identifier == "Goal and Shot Creation ":
        RepeatedColumn = ['PassLive','PassDead','TO','Sh','Fld','Def']
        renameduplicatecolumns(RepeatedColumn, all_squad_record_df)
        sourcedestcompare('PlayerChancesStat', all_squad_record_df)
    if html_identifier == "Defensive Actions ":
        RepeatedColumn = ['Tkl']
        renameduplicatecolumns(RepeatedColumn, all_squad_record_df)    
        sourcedestcompare('PlayerDefensiveactionStat', all_squad_record_df)
    
    
    # Send to database (if i want to replace table add if_exists='replace' to the parameter; if_exists='append' )
    all_squad_record_df["Round"] = Last_Round
    # if you want to export to dbo then remove the schema parameter
    # all_squad_record_df.to_sql(database_table, engine, if_exists='append'#, schema='sch98'
    #                    )

    try:
        all_squad_record_df.to_sql(database_table, engine, if_exists= Data_Base_Push_Method) # Add schema='sch98' if needed
        print(f"{Fore.GREEN}Table {database_table} has been updated with {len(all_squad_record_df)} records{Style.RESET_ALL}")
    except SQLAlchemyError as e:
        # Extract the error message
        error_message = str(e.__cause__)
        
        if 'Invalid column name' in error_message or 'column name or number of supplied values does not match' in error_message:
            # Log the column mismatch error
            print(f"{Fore.RED}Column mismatch error when inserting data into table '{database_table}': {error_message}{Style.RESET_ALL}")
        else:
            # Log any other SQLAlchemy errors
            print(f"{Fore.RED}An error occurred: {error_message}{Style.RESET_ALL}")

### troubleshooting for limit violation

In [6]:
# standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# data = requests.get(standing_url)
# # data

In [7]:
# soup = BeautifulSoup(data.text, 'lxml')
# soup

### Executable: player_record_to_database

In [8]:
# Import Table Parameter:
csv_file = 'player_record_database_name.csv'
player_array_of_tuples = read_csv_to_array_of_tuples(csv_file)

In [9]:
for html_identifier, database_table in player_array_of_tuples:
    print(f"Begin {database_table} database")
    player_record_to_database(html_identifier, database_table)
    # pause for 1mins before iterating
    time.sleep(30)

Begin PlayerGoalkeepingStat database
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats processing
Table completed: https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats processing
Table completed: https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats processing
Table completed: https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
https://fbref.com/en/squads/18bb7c10/Arsenal-Stats processing
Table completed: https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats processing
Table completed: https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats processing
Table completed: https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats processing
Table completed: https://fbref.com/en/squads/d07537b9/Brig

C:\Users\User\AppData\Local\Temp\ipykernel_35784\2933469533.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sqlcolumnlist = pd.read_sql(sql_columnlist, conn)


['Round']
[]
Table PlayerPassingStat has been updated with 425 records
Begin PlayerStandardStat database
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats processing
Table completed: https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats processing
Table completed: https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats processing
Table completed: https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
https://fbref.com/en/squads/18bb7c10/Arsenal-Stats processing
Table completed: https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats processing
Table completed: https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats processing
Table completed: https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats p

C:\Users\User\AppData\Local\Temp\ipykernel_35784\2933469533.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sqlcolumnlist = pd.read_sql(sql_columnlist, conn)


['Round']
['PrgC', 'PrgP', 'PrgR']
Table PlayerStandardStat has been updated with 511 records
Begin PlayerShootingStat database
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats processing
Table completed: https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats processing
Table completed: https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats processing
Table completed: https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
https://fbref.com/en/squads/18bb7c10/Arsenal-Stats processing
Table completed: https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats processing
Table completed: https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats processing
Table completed: https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
https://fbref.com/en/squads/d07537b9/Brighton-

C:\Users\User\AppData\Local\Temp\ipykernel_35784\2933469533.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sqlcolumnlist = pd.read_sql(sql_columnlist, conn)


['Round']
[]
Table PlayerChancesStat has been updated with 425 records
Begin PlayerDefensiveactionStat database
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats processing
Table completed: https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats processing
Table completed: https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats processing
Table completed: https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
https://fbref.com/en/squads/18bb7c10/Arsenal-Stats processing
Table completed: https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats processing
Table completed: https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats processing
Table completed: https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-

C:\Users\User\AppData\Local\Temp\ipykernel_35784\2933469533.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sqlcolumnlist = pd.read_sql(sql_columnlist, conn)


['Round']
[]
Table PlayerDefensiveactionStat has been updated with 425 records
Begin PlayerPossessionStat database
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats processing
Table completed: https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats processing
Table completed: https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats processing
Table completed: https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
https://fbref.com/en/squads/18bb7c10/Arsenal-Stats processing
Table completed: https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats processing
Table completed: https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats processing
Table completed: https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albi

## Squad Function: squad_record_to_database


In [10]:
def squad_record_to_database(html_identifier, database_table):
    data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
    squad_record = pd.read_html(data.text, match = html_identifier)[0]
    squad_record.columns = squad_record.columns.droplevel()
    if html_identifier == "Squad Goalkeeping ":
        RepeatedColumn = ['Save%']
        return RepeatedColumn
    if html_identifier == "Squad Advanced Goalkeeping ":
        RepeatedColumn = ['Att', 'Launch%', 'AvgLen']
        return RepeatedColumn
    if html_identifier == "Squad Passing ":
        RepeatedColumn = ['Cmp','Att','Cmp%']
        return RepeatedColumn
    if html_identifier == "Squad Standard Stats ":
        RepeatedColumn = ['Gls','Ast','xG','xAG']
        renameduplicatecolumns(RepeatedColumn, squad_record)
        return RepeatedColumn
    if html_identifier == "Goal and Shot Creation ":
        RepeatedColumn = ['PassLive','PassDead','TO','Sh','Fld','Def']
        return RepeatedColumn
    if html_identifier == "Defensive Actions ":
        RepeatedColumn = ['Tkl']
        return RepeatedColumn 
    
    if html_identifier in("Squad Goalkeeping ", "Squad Advanced Goalkeeping ", "Squad Passing ", "Squad Standard Stats ", "Goal and Shot Creation ", "Defensive Actions "):
        renameduplicatecolumns(RepeatedColumn, squad_record)
    if html_identifier == "Squad Standard Stats ":
        squad_record = squad_record.drop(['PrgC','PrgP'],axis=1)
    squad_record["Season"] = "2024/2025"
    squad_record["Round"] = Last_Round

    try:
        squad_record.to_sql(database_table, engine, if_exists= Data_Base_Push_Method#, schema='sch98'
                           )
        print(f"{Fore.GREEN}Table completed: {html_identifier}{Style.RESET_ALL}")
    except SQLAlchemyError as e:
        # Extract the error message
        error_message = str(e.__cause__)
        
        if 'Invalid column name' in error_message or 'column name or number of supplied values does not match' in error_message:
            # Log the column mismatch error
            print(f"{Fore.RED}Column mismatch error when inserting data into table '{database_table}': {error_message}{Style.RESET_ALL}")
            squad_record.to_sql(database_table, engine, if_exists='replace', schema='mismatch')
            print(f"{Fore.RED}Uploaded '{database_table}' to mismatch schema{Style.RESET_ALL}")
        else:
            # Log any other SQLAlchemy errors
            print(f"{Fore.RED}An error occurred: {error_message}{Style.RESET_ALL}")

### Executable: squad_record_to_database

In [11]:
# Example usage:
csv_file_squad = 'squad_record_database_name.csv'
squad_array_of_tuples = read_csv_to_array_of_tuples(csv_file_squad)

In [12]:
for html_identifier, database_table in squad_array_of_tuples:
    print(f"Begin {database_table} database")
    squad_record_to_database(html_identifier, database_table)
    # pause for 1mins before iterating
    time.sleep(30)

Begin SquadGoalkeepingStat database
Table completed: Squad Goalkeeping
Begin SquadAdvancedGoalkeepingStat database
Table completed: Squad Advanced Goalkeeping
Begin SquadPassingStat database
Table completed: Squad Passing
Begin SquadStandardStat database
Table completed: Squad Standard Stats
Begin SquadShootingStat database
Table completed: Shooting
Begin SquadPasstypeStat database
Table completed: Pass Types
Begin SquadChancesStat database
Table completed: Goal and Shot Creation
Begin SquadDefensiveactionStat database
Begin SquadPossessionStat database
Table completed: Possession 
Begin SquadPlayingtimeStat database
Table completed: Playing Time 
Begin SquadMiscellaneousStat database
Table completed: Miscellaneous Stats 


# Total goals and assist contribution for the last 5 matches

In [13]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [14]:

# team_urls = [l for l in team_urls if l not in ['https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats','https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats','https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats','https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats','https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats','https://fbref.com/en/squads/fd962109/Fulham-Stats','https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats']]


all_PlayerLast5Performance = []


for team_url in (url for url in team_urls):
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    print(f"Begin {team_name} Last 5 performance")
# team_url = team_urls[0]
# team_url
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text, 'lxml')
    scorefixture_table = soup.select('table.stats_table')[1]
    links = scorefixture_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/matches' in l ]
    # remove duplicate from list
    links = list(dict.fromkeys(links))
    # removing non premier league (Remove europa)
    links = [l for l in links if "Premier" in l]
    links
    
    
    last5_marchreport = []
    n = -1    
    while n >-6:
        # check, especially within the first 5matches of the season, that n should be within the index range
        if n >= -len(links):  # Check if n is within the valid index range
            data = requests.get(f"https://fbref.com{links[n]}")
            summary = pd.read_html(data.text, match= mapping[team_name]+" Player Stats Table")[0]
            summary.columns = summary.columns.droplevel()
            # retrun the opposition team name
            string = links[n]
            modifiedstring = string[string.rindex('/') + 1:]
            words = re.findall(r'\b\w+\b', modifiedstring)
            # Remove the last 5 words
            summary["Clash"] = ' '.join(words[:-5])



            summary["Club_"] = team_name
            #create a short name
            summary["ShClub_"] = mapping[team_name]
            summary.drop(summary.tail(1).index, inplace=True)
        # if it is out of range then move to the next loop
        else:
            n -= 1
            continue
        
        n -= 1
        
        last5_marchreport.append(summary)
        time.sleep(15)

    # ===================================================================================================================
    #  Bournemouth and Luton Town giving some errors but have been excepted
    # ===================================================================================================================
    try:
        pd_summary = pd.concat(last5_marchreport)
    except pd.errors.InvalidIndexError as e:
        print(f"{Fore.RED}Error: {e}{Style.RESET_ALL}")
        print("Reindexing is only valid with uniquely valued Index objects.")
        print(f"{Fore.RED}Team: {team_name} thus skipped{Style.RESET_ALL}")
        
    
    all_PlayerLast5Performance.append(pd_summary)
    time.sleep(15)

pd_allteam = pd.concat(all_PlayerLast5Performance)

Begin Manchester City Last 5 performance
Begin Liverpool Last 5 performance
Begin Aston Villa Last 5 performance
Begin Arsenal Last 5 performance
Begin Chelsea Last 5 performance
Begin Newcastle United Last 5 performance
Begin Brighton and Hove Albion Last 5 performance
Begin Nottingham Forest Last 5 performance
Begin Fulham Last 5 performance
Begin Tottenham Hotspur Last 5 performance
Begin Manchester United Last 5 performance
Begin Brentford Last 5 performance
Begin Bournemouth Last 5 performance
Begin West Ham United Last 5 performance
Begin Leicester City Last 5 performance
Begin Crystal Palace Last 5 performance
Begin Ipswich Town Last 5 performance
Begin Southampton Last 5 performance
Begin Everton Last 5 performance
Begin Wolverhampton Wanderers Last 5 performance


In [15]:
# pd_allteam.columns

In [19]:
# Send the granula data for analysis the games where a player socred
dfperformanceagaisntclub = pd_allteam[['Player','#','Min','Gls','Ast','Sh','SoT','CrdY','CrdR','Blocks', 'Tkl', 'Int','SCA','GCA','Club_','ShClub_', 'Clash']]
dfperformanceagaisntclub["Season"] = "2024/2025"
dfperformanceagaisntclub["Round"] = Last_Round
dfperformanceagaisntclub.to_sql('_PlayerPerformanceRecentGran', engine, if_exists= Data_Base_Push_Method #, schema='sch98'
                       )
# dfperformanceagaisntclub.to_sql('_Debug-PlayerPerformanceRecentGran', engine, if_exists='append')

C:\Users\User\AppData\Local\Temp\ipykernel_35784\4053840249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfperformanceagaisntclub["Season"] = "2024/2025"
C:\Users\User\AppData\Local\Temp\ipykernel_35784\4053840249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfperformanceagaisntclub["Round"] = Last_Round


-1

In [20]:
dfperformancerecent = pd_allteam[['Player','#','Min','Gls','Ast','Sh','SoT','CrdY','CrdR','Blocks', 'Tkl', 'Int','SCA','GCA','Club_','ShClub_']]
dfperformancerecent = dfperformancerecent.groupby(['#','Player','Club_', 'ShClub_']).sum().reset_index()



dfperformancerecent["Season"] = "2024/2025"



In [21]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
dfperformancerecent["Round"] = Last_Round
dfperformancerecent.to_sql('_PlayerPerformanceRecent', engine, if_exists= Data_Base_Push_Method #, schema='sch98'
                          )
# dfperformancerecent.to_sql('_Debug-PlayerPerformanceRecent', engine, if_exists='append')

-1

# Team Result

In [22]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_TeamresultStat = []

In [23]:
# for team_url in (url for url in team_urls if url == "https://fbref.com/en/squads/fd962109/Fulham-Stats"):
for team_url in (url for url in team_urls):
    print(f"Begin {team_url}")
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    ResultStat = pd.read_html(data.text, match = "Scores & Fixtures ")[0]
#     ResultStat.columns = ResultStat.columns.droplevel()
    ResultStat["Club"] = team_name
    ResultStat["ShClub_"] = mapping[team_name]
#     ResultStat.drop(ResultStat.tail(2).index, inplace=True)

    all_TeamresultStat.append(ResultStat)
    time.sleep(15)    
    

ResultStat_df = pd.concat(all_TeamresultStat)  

Begin https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Begin https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Begin https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
Begin https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
Begin https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Begin https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
Begin https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats
Begin https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats
Begin https://fbref.com/en/squads/fd962109/Fulham-Stats
Begin https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats
Begin https://fbref.com/en/squads/19538871/Manchester-United-Stats
Begin https://fbref.com/en/squads/cd051869/Brentford-Stats
Begin https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats
Begin https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats
Begin https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats
Begin https://fbref.com/en/squads/47c64c55/Crystal-Pala

In [24]:
ResultStat_df = ResultStat_df[ResultStat_df['Result'].notna()]
ResultStat_df = ResultStat_df[ResultStat_df['Comp']=='Premier League']

In [25]:
ResultStat_df["Season"] = "2024/2025"

In [26]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
ResultStat_df =ResultStat_df.rename({"Round":"MatchWeek"}, axis='columns')
ResultStat_df["Round"] = Last_Round
ResultStat_df.to_sql('FixturesStat', engine, if_exists= Data_Base_Push_Method #, schema='sch98'
                       )

-1

# Table Standings

In [27]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
Standingtable = pd.read_html(data.text, match = "Premier League Table")[0]
# Standingtable[Standingtable['Squad']]

In [28]:
Standingtable_df = Standingtable[['Rk','Squad','W','D','L','Pts','GF','GA','GD','Last 5','Top Team Scorer','MP']]

In [29]:
Standingtable_df["Season"] = "2024/2025"

C:\Users\User\AppData\Local\Temp\ipykernel_35784\1238377272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Standingtable_df["Season"] = "2024/2025"


In [30]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
Standingtable_df["Round"] = Last_Round
Standingtable_df.to_sql('_Standtable', engine, if_exists= Data_Base_Push_Method #, schema='sch98'
                       )

C:\Users\User\AppData\Local\Temp\ipykernel_35784\1602906079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Standingtable_df["Round"] = Last_Round


-1

# Next 7 fixtures

In [31]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
# team_urls = [l for l in team_urls if l not in ['https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats','https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats','https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats','https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats','https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats','https://fbref.com/en/squads/fd962109/Fulham-Stats','https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats']]


all_Next7Fixture = []


for team_url in (url for url in team_urls):
    print(f"Begin {team_url}")
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
#     team_url = team_urls[0]
    # team_url
    data = requests.get(team_url)
    fixtures = pd.read_html(data.text, match= "Scores & Fixtures ")[0]
    fixtures["shClub"] = mapping[team_name]
    fixtures = fixtures[fixtures['Comp']=='Premier League']
    fixtures[['Match Report']] = fixtures[['Match Report']].fillna('')
    fixtures = fixtures[fixtures['Match Report'].str.contains('Match Report')==False]
    
    fixtures = fixtures.iloc[:7]
# converting object type to date data type
    #     fixtures["Date"] = pd.to_datetime(fixtures["Date"])
#     fixtures.nsmallest(n=7, columns=['Date'])
    
    all_Next7Fixture.append(fixtures)
    time.sleep(30)
    
pd_allNext7Fixtures = pd.concat(all_Next7Fixture)
# pd_allNext7Fixtures  
    


Begin https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Begin https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Begin https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
Begin https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
Begin https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Begin https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
Begin https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats
Begin https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats
Begin https://fbref.com/en/squads/fd962109/Fulham-Stats
Begin https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats
Begin https://fbref.com/en/squads/19538871/Manchester-United-Stats
Begin https://fbref.com/en/squads/cd051869/Brentford-Stats
Begin https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats
Begin https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats
Begin https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats
Begin https://fbref.com/en/squads/47c64c55/Crystal-Pala

In [32]:
pd_allNext7Fixtures["Season"] = "2024/2025"

In [33]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
pd_allNext7Fixtures["Round"] = Last_Round
pd_allNext7Fixtures.to_sql('Next7Fixtures', engine, if_exists= Data_Base_Push_Method #, schema='sch98'
                       )

-1

# This week and Next week Fixture draw

#### Edit filter condition within the first block of code

<font color='blue'>Edit filter condition within the first block of code</font>

In [34]:
nextweekfixture_url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
data = requests.get(nextweekfixture_url)
nextfixtures = pd.read_html(data.text, match= "Scores & Fixtures ")[0]
nextfixtures = nextfixtures[(nextfixtures["Match Report"]=="Head-to-Head")&((nextfixtures["Wk"]==Next_Week)|(nextfixtures["Wk"]==Upper_Week))]
# #requires that the column changes field type : I was changing from float to string. 
# Week = {1.0: 'This week', 2.0: 'Next week'}
# nextfixtures = nextfixtures.replace({"Wk":Week})
# #creating a new column entirely
nextfixtures['Week'] = ['This week' if x==Next_Week else 'Next week' for x in nextfixtures['Wk']]
nextfixtures

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Week
55,6.0,Sat,2024-09-28,12:30,Newcastle Utd,NaN,NaN,NaN,Manchester City,NaN,St James' Park,NaN,Head-to-Head,NaN,This week
56,6.0,Sat,2024-09-28,15:00,Chelsea,NaN,NaN,NaN,Brighton,NaN,Stamford Bridge,NaN,Head-to-Head,NaN,This week
57,6.0,Sat,2024-09-28,15:00,Arsenal,NaN,NaN,NaN,Leicester City,NaN,Emirates Stadium,NaN,Head-to-Head,NaN,This week
58,6.0,Sat,2024-09-28,15:00,Brentford,NaN,NaN,NaN,West Ham,NaN,Gtech Community Stadium,NaN,Head-to-Head,NaN,This week
59,6.0,Sat,2024-09-28,15:00,Everton,NaN,NaN,NaN,Crystal Palace,NaN,Goodison Park,NaN,Head-to-Head,NaN,This week
60,6.0,Sat,2024-09-28,15:00,Nott'ham Forest,NaN,NaN,NaN,Fulham,NaN,The City Ground,NaN,Head-to-Head,NaN,This week
61,6.0,Sat,2024-09-28,17:30,Wolves,NaN,NaN,NaN,Liverpool,NaN,Molineux Stadium,NaN,Head-to-Head,NaN,This week
62,6.0,Sun,2024-09-29,14:00,Ipswich Town,NaN,NaN,NaN,Aston Villa,NaN,Portman Road Stadium,NaN,Head-to-Head,NaN,This week
63,6.0,Sun,2024-09-29,16:30,Manchester Utd,NaN,NaN,NaN,Tottenham,NaN,Old Trafford,NaN,Head-to-Head,NaN,This week
64,6.0,Mon,2024-09-30,20:00,Bournemouth,NaN,NaN,NaN,Southampton,NaN,Vitality Stadium,NaN,Head-to-Head,NaN,This week


In [35]:
nextfixtures["Season"] = "2024/2025"

In [36]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
nextfixtures["Round"] = Last_Round
nextfixtures.to_sql('NextweekFixtures', engine, if_exists= Data_Base_Push_Method #, schema='sch98'
                       )

-1